### importes

In [1]:
import pandas as pd
import numpy as np
import sklearn as nk

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('data/titanic.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [5]:
df.shape

(891, 12)

In [6]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
count = pd.DataFrame(df['Survived'].value_counts())
percent = pd.DataFrame(df['Survived'].value_counts(normalize=True) * 100)
table = pd.concat([count, percent], axis = 1)
table.columns = ['#Target', '%Target']
table

,#Target,%Target
0,549,61.616162
1,342,38.383838


#### Checando valores nulos

In [8]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

#### Correlação entre as variaveis categoricas

In [9]:
df.iloc[:,2:].corr()

,Pclass,Age,SibSp,Parch,Fare
Pclass,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,-0.549500,0.096067,0.159651,0.216225,1.000000


#### Tratando valores nulos

In [10]:
df['Age'].fillna(df['Age'].mean(), inplace=True)
df['Cabin'].fillna('vazio', inplace=True)
df['Embarked'].fillna('vazio', inplace=True)

#### Transformar as váriaveis categóricas em numéricas nossa váriavel resposta para forma binária

In [11]:
categ_var = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Cabin', 'Embarked']
for i in categ_var:
    print(f'Coluna {i}')
    print(df[i].value_counts())
    print('=================================')
    print('=================================')

Coluna Pclass
3    491
1    216
2    184
Name: Pclass, dtype: int64
Coluna Sex
male      577
female    314
Name: Sex, dtype: int64
Coluna Age
29.699118    177
24.000000     30
22.000000     27
18.000000     26
28.000000     25
            ... 
55.500000      1
53.000000      1
20.500000      1
23.500000      1
0.420000       1
Name: Age, Length: 89, dtype: int64
Coluna SibSp
0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64
Coluna Parch
0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64
Coluna Fare
8.0500     43
13.0000    42
7.8958     38
7.7500     34
26.0000    31
           ..
8.4583      1
9.8375      1
8.3625      1
14.1083     1
17.4000     1
Name: Fare, Length: 248, dtype: int64
Coluna Cabin
vazio          687
B96 B98          4
G6               4
C23 C25 C27      4
C22 C26          3
              ... 
C49              1
C50              1
B50              1
C86              1
C45              1
Name:

In [12]:
def t_sex(col):
    if col == 'male':
        return 0
    elif col == 'female':
        return 1
    
def t_cabin(col):
    if col == 'vazio':
        return 0
    elif col == 'B96 B98':
        return 1
    elif col == 'G6':
        return 2
    elif col == 'C23 C25 C27':
        return 3
    elif col == 'C22 C26':
        return 4
    else:
        return 5

def t_embarked (col):
    if col == 'S':
        return 0
    elif col == 'C':
        return 1
    else:
        return 2

In [14]:
df['Sex'] = df['Sex'].apply(t_sex)
df['Cabin'] = df['Cabin'].apply(t_cabin)
df['Embarked'] = df['Embarked'].apply(t_embarked)
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.000000,1,0,A/5 21171,7.2500,0,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.000000,1,0,PC 17599,71.2833,5,1
2,3,1,3,"Heikkinen, Miss. Laina",1,26.000000,0,0,STON/O2. 3101282,7.9250,0,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.000000,1,0,113803,53.1000,5,0
4,5,0,3,"Allen, Mr. William Henry",0,35.000000,0,0,373450,8.0500,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",0,27.000000,0,0,211536,13.0000,0,0
887,888,1,1,"Graham, Miss. Margaret Edith",1,19.000000,0,0,112053,30.0000,5,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",1,29.699118,1,2,W./C. 6607,23.4500,0,0
889,890,1,1,"Behr, Mr. Karl Howell",0,26.000000,0,0,111369,30.0000,5,1


#### Selecionando as variveis

In [18]:
filter_data = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Cabin', 'Embarked']]
target_data = df['Survived']

## Sepear Treino de Teste

In [24]:
X_all = filter_data
y_all = target_data

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.20, random_state= 100000)

In [61]:
print(f'Número de observações na base de Treino: {len(X_train)}')
print(f'Número de observações na base de teste: {len(X_test)}')

Número de observações na base de Treino: 712
Número de observações na base de teste: 179


In [62]:
print(f'Shape base Treino: {X_train.shape}')
print(f'Shape base Teste: {X_test.shape}')

Shape base Treino: (712, 8)
Shape base Teste: (179, 8)


In [63]:
print(f'Shape resposta Teste: {len(y_train)}')
print(f'Shape resposta Treino: {len(y_test)}')

Shape resposta Teste: 712
Shape resposta Treino: 179


## Modelagem e validação dos resultados

#### Treinar uma DecisionTreeClassifier

In [64]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [125]:
clf = DecisionTreeClassifier(min_samples_leaf=20, max_depth=3, random_state=123)

In [126]:
modelo = clf.fit(X_train, y_train)

#### Validação cruzada com 5 folds e verificar a acurácia para cada fold

In [127]:
scores = cross_val_score(clf, X_train, y_train, cv=5)
scores

array([0.77622378, 0.73426573, 0.79577465, 0.81690141, 0.76056338])

In [128]:
print(f'Accuracy: {scores.mean() :.2f}(+/- {scores.std() :.2f})')

Accuracy: 0.78(+/- 0.03)


#### Predicendo na base de teste  e calculando acc

In [129]:
y_predito = modelo.predict_proba(X_test)
y_predito

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.41666667, 0.58333333],
       [0.42857143, 0.57142857],
       [0.86834734, 0.13165266],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.86834734, 0.13165266],
       [0.41666667, 0.58333333],
       [0.09090909, 0.90909091],
       [0.41666667, 0.58333333],
       [0.46296296, 0.53703704],
       [0.46296296, 0.53703704],
       [0.86834734, 0.13165266],
       [0.86834734, 0.13165266],
       [0.86834734, 0.13165266],
       [0.86834734, 0.13165266],
       [0.86834734, 0.13165266],
       [0.41666667, 0.58333333],
       [0.46296296, 0.53703704],
       [0.09090909, 0.90909091],
       [0.        , 1.        ],
       [0.41666667, 0.58333333],
       [0.75      , 0.25      ],
       [0.09090909, 0.90909091],
       [0.86834734, 0.13165266],
       [0.86834734, 0.13165266],
       [0.86834734, 0.13165266],
       [0.86834734, 0.13165266],
       [0.86834734, 0.13165266],
       [0.

In [130]:
y_predito = modelo.predict(X_test)
y_predito

array([1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0], dtype=int64)

In [131]:
acc_test = accuracy_score(y_test, y_predito)
acc_test

0.8212290502793296